In [ ]:
#简化版市场法估值程序

In [ ]:
#导入库
import pandas as pd
import akshare as ak
import datetime

In [ ]:
##申万行业分类表格提取
basestation ="swclass217.xlsx"
###pandas库提取excel命令.read_excel
data = pd.read_excel(basestation)
data

In [ ]:
##设置基准日和待查上市公司名
b_date='20230228'
c_name='双汇发展'

In [ ]:
##pandas用&进行多条件'与'筛选行业名称及A股同行业公司
###选取行业
ind_name=data[(data['公司简称']==c_name)&(data['交易所']=='A股')]['新版三级行业']
###重置序号
ind_name.index=[0]
###筛选同行业公司
data[(data['新版三级行业']==ind_name[0])&(data['交易所']=='A股')]

In [ ]:
##取行业中位数乘数
###建立空存储列表
industry_ps_list=[]
industry_pe_list=[]
industry_pb_list=[]
###调整日期格式匹配市盈率库
b_date1=datetime.date(int(b_date[0:4]), int(b_date[4:6]), int(b_date[6:8]))
###批量拉取数据并存入列表
for i in data[(data['新版三级行业']==ind_name[0])&(data['交易所']=='A股')]['股票代码']:
#清洗股票代码和日期格式为提取格式对应
    b=i.split('.')
    i=b[0]
    stock_a_lg_indicator_df = ak.stock_a_lg_indicator(symbol=i)
####将负市盈率填充为-1
    indu_df=stock_a_lg_indicator_df.fillna(-1)    
    industry_ps_list.append(indu_df[indu_df['trade_date']==b_date1]['ps_ttm'].values[0])
    industry_pe_list.append(indu_df[indu_df['trade_date']==b_date1]['pe_ttm'].values[0])
    industry_pb_list.append(indu_df[indu_df['trade_date']==b_date1]['pb'].values[0])
industry_temp=pd.DataFrame()
industry_temp['ps_ttm']=industry_ps_list
industry_temp['pe_ttm']=industry_pe_list
industry_temp['pb']=industry_pb_list
#print(industry_temp)
####使用pandas的median函数计算行业乘数中位数
industry_temp_ps=industry_temp['ps_ttm'].median()
industry_temp_pe=industry_temp['pe_ttm'].median() 
industry_temp_pb=industry_temp['pb'].median() 
industry_temp_ps,industry_temp_pe,industry_temp_pb

In [ ]:
##各指标乘数估值测算
###拉取个股数据
###公司名称->代码
stock_data = ak.stock_zh_a_spot_em()
c_code=stock_data[stock_data["名称"]==c_name].iloc[0]["代码"]
###收盘价close,ps_ttm,pe_ttm,pb
stock_data = ak.stock_zh_a_hist(symbol=c_code, period="daily", start_date=b_date, end_date=b_date, adjust="qfq")
stock_price = stock_data.iloc[0]["收盘"]
rate_df = ak.stock_a_lg_indicator(symbol=i)
df = pd.DataFrame({
    "股票名称": [c_name],
    "股票代码": [c_code],
    "基准日": [b_date],
    "close": [stock_price],
    "ps_ttm": [rate_df[rate_df['trade_date']==b_date1]['ps_ttm'].values[0]],
    "pe_ttm": [rate_df[rate_df['trade_date']==b_date1]['pe_ttm'].values[0]],
    "pb": [rate_df[rate_df['trade_date']==b_date1]['pb'].values[0]]
})
###各指标乘数估值测算
price_ps=df['close'][0]*industry_temp_ps/df['ps_ttm'][0]
price_pe=df['close'][0]*industry_temp_pe/df['pe_ttm'][0]
price_pb=df['close'][0]*industry_temp_pb/df['pb'][0]
###异常处理
if price_ps<0:
    price_ps=0
if price_pe<0:
    price_pe=0
if price_pb<0:
    price_pb=0

##分行业类型和是否科创板测算市场法估值
###给出科创板、传统行业、其他行业权重

mv_weight_kcb=[]
mv_weight_tradi=[]
mv_weight_else=[0.2,0.5,0.3]
if industry_temp_pe<0:
    mv_weight_kcb=[]
    mv_weight_tradi=[]
    mv_weight_else=[0.2,0,0.8]      

###兜底估值
mv_weight=[0,0,1]
mv_basic=mv_weight[0]*price_ps+mv_weight[1]*price_pe+mv_weight[2]*price_pb
###分行业估值
###这里简化判断行业部分，请同学们自行设定分类
industry_def='传统行业'
if industry_def=='科创板':
    mv_weight=mv_weight_kcb
    mv=mv_weight[0]*price_ps+mv_weight[1]*price_pe+mv_weight[2]*price_pb
elif industry_def=='传统行业':
    mv=mv_basic
else:
    mv_weight=mv_weight_else
    mv=mv_weight[0]*price_ps+mv_weight[1]*price_pe+mv_weight[2]*price_pb
if mv<mv_basic:
    mv=mv_basic

print(c_name+b_date+'日股价为'+str(df['close'][0])+'元,'+'市场法估值结果为'+str(round(mv,2))+'元')

In [ ]:
#相比个人软著的简化部分，peg等其他加权指标，智能行业分类，数据库连接，更多的异常处理

In [ ]:
#生成可视化图表

In [ ]:
#导入图表对应库
import matplotlib.pyplot as plt

In [ ]:
# 数据可视化
##.figure创建图像
fig=plt.figure()
##显示中文字体(.rcParams)参数设置命令
plt.rcParams['font.sans-serif'] = ['SimHei']  
##处理负号显示
plt.rcParams['axes.unicode_minus'] = False  
##估价图
label_list = [df['close'][0],round(mv,2)]
num_list1 = [float(df['close'][0])]      
num_list2 = [float(round(mv,2))]      


"""
绘制条形图（.bar）
起点位置
height:长条形高度
width:长条形宽度
label:为后面设置legend准备
"""
rects1 = plt.bar(0, height=num_list1, width=0.4, color='red', label="当前股价")
rects2 = plt.bar(0.4, height=num_list2, width=0.4, color='aqua', label="市场比较估价")
"""
设置x轴刻度显示值(.xticks)
参数一：中点坐标
参数二：显示值
"""
plt.xticks([0,0.4], label_list)
plt.title(f"{c_name}简式市场比较估价")
## 设置题注
plt.legend()     

##显示图片
plt.show()
##关闭画图
plt.close()

In [ ]:
#生成excel

In [ ]:
#导入excel对应库
import xlwings as xw

In [ ]:
##创建Excel文件，并设置为不可见
app = xw.App(visible=False)
wb = app.books.add()
##创建excel工作表
sht = wb.sheets.add('project')
##写内容(.range位置.value内容，format字符串中加函数简化为f+{})
sht.range(1,1).value = f'使用市场比较估价法对{c_name}公司进行估价分析.\n\n\
(仅供参考，不作为投资依据)'
sht.range(2,1).value = f'数据来源:Tushare'
sht.range(3,1).value = f'日期:{b_date}'
price=df['close'][0]
sht.range(4,1).value = f'股价为{price}元,市场法估值结果为{str(round(mv,2))}元'
##调整格式
###.column_width宽
sht.range("A1:G100").column_width = 25
###.api.WrapText文字自动换行
sht.range("A1:A4").api.WrapText = True
###合并单元格.merge
sht.range("A1:B1").merge()
sht.range("A4:B4").merge()
###.row_height行高
sht.range("A1").row_height = 50
###设置字体边框
sht.range("A1:G100").api.Font.Size = 12          # 设置字体的大小。
sht.range("A1:G100").api.Font.Bold = True        # 设置为粗体。
sht.range("A1:B4").api.Borders(7).LineStyle = 1
sht.range("A1:B4").api.Borders(8).LineStyle = 1
sht.range("A1:B4").api.Borders(9).LineStyle = 1
sht.range("A1:B4").api.Borders(10).LineStyle = 1
###插入图片.pictures.add
sht.pictures.add(fig, name='图1', update=True, left=400)
###文件储存
wb.save(f'{c_name}估价报告.xlsx')
###关闭excel链接
wb.close()
app.quit()

In [ ]:
#生成word

In [ ]:
#导入word对应库
##pip install python-docx
import docx
##创建word文件
file = docx.Document()
##设置居中.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
from docx.enum.text import WD_ALIGN_PARAGRAPH
##添加标题
file.add_heading(f'{c_name}估价报告', level=0).paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
##添加文字.add_paragraph
file.add_paragraph(f'使用市场比较估价法对{c_name}公司进行估价分析.').paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
file.add_paragraph('(仅供参考，不作为投资依据)').paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
file.add_paragraph('数据来源:Tushare').paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
file.add_paragraph(f'股价为{price}元,市场法估值结果为{str(round(mv,2))}元').paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
###插入图片.addpictures
file.add_picture('temp.jpg')
###文件储存.save
file.save(f'{c_name}估价报告.docx') 